In [17]:
# 구글드라이브 import
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
import os
import sys

os.environ['MSPN_HOME'] = '/content/drive/MyDrive/Colab Notebooks/MSPN'

# PYTHONPATH 설정
os.environ['PYTHONPATH'] = f"{os.environ.get('PYTHONPATH', '')}:{os.environ['MSPN_HOME']}"

# 환경 변수 및 경로 설정 확인
print(f"MSPN_HOME: {os.environ['MSPN_HOME']}")
print(f"PYTHONPATH: {os.environ['PYTHONPATH']}")

MSPN_HOME: /content/drive/MyDrive/Colab Notebooks/MSPN
PYTHONPATH: /env/python:/content/drive/MyDrive/Colab Notebooks/mspn:/content/drive/MyDrive/Colab Notebooks/MSPN


In [19]:
!pip3 install torch
!pip3 install torchvision

In [20]:
if not os.path.exists('/content/drive/MyDrive/Colab Notebooks'):
  os.makedirs('/content/drive/MyDrive/Colab Notebooks')
os.chdir('/content/drive/MyDrive/Colab Notebooks')

In [21]:
!git clone https://github.com/megvii-research/MSPN

fatal: destination path 'MSPN' already exists and is not an empty directory.


In [22]:
os.chdir('/content/drive/MyDrive/Colab Notebooks/MSPN')

In [23]:
!pip3 install -r requirements.txt

In [24]:
!pip3 install -U git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI

  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-mdmcru6j
  Running command git clone --filter=blob:none --quiet https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-mdmcru6j
  Resolved https://github.com/cocodataset/cocoapi.git to commit 8c9bcc3cf640524c4c20a9c40e89cb6a2f2fa0e9
  Preparing metadata (setup.py) ... done


In [25]:
!cp ./exps/mspn.2xstg.coco/* .

In [26]:
import argparse
import os
import os.path as osp
import time

from cvpack.utils.pyt_utils import (
    parse_torch_devices, extant_file, link_file, ensure_dir)
from cvpack.utils.logger import get_logger
import torch
from tensorboardX import SummaryWriter

from cvpack.torch_modeling.engine.engine import State
from cvpack.utils.pyt_utils import ensure_dir

from config import cfg
from network import MSPN
from lib.utils.dataloader import get_train_loader
from lib.utils.solver import make_lr_scheduler, make_optimizer
from collections import OrderedDict
from cvpack.utils.pyt_utils import (parse_torch_devices, extant_file, link_file, ensure_dir)
from cvpack.utils.logger import get_logger
from cvpack.torch_modeling.engine.checkpoint import load_model

In [27]:
parser = argparse.ArgumentParser()

In [32]:
class Engine(object):
    def __init__(self, cfg, custom_parser=None):
        self.version = 0.01
        self.state = State()
        self.devices = '0'
        self.distributed = False
        self.logger = None
        self.cfg = cfg

        if custom_parser is None:
            self.parser = argparse.ArgumentParser()
        else:
            assert isinstance(custom_parser, argparse.ArgumentParser)
            self.parser = custom_parser

        #self.inject_default_parser()
        #self.args = self.parser.parse_args(args=[])

        self.continue_state_object = 'exps'

        if 'WORLD_SIZE' in os.environ:
            self.distributed = int(os.environ['WORLD_SIZE']) > 1


        self.world_rank = 1
        self.devices = parse_torch_devices('*')

    def setup_log(self, name='train', log_dir=None, file_name=None):
        if not self.logger:
            self.logger = get_logger(
                name, log_dir, 0, filename=file_name)
        else:
            self.logger.warning('already exists logger')
        return self.logger

    def inject_default_parser(self):
        p = self.parser
        p.add_argument(
            '-d', '--devices', default='0',
            help='set data parallel training')
        p.add_argument(
            '-c', '--continue', type=extant_file, metavar="FILE",
            dest="continue_fpath",
            help='continue from one certain checkpoint')
        p.add_argument(
            '--local_rank', default=0, type=int,
            help='process rank on node')

    def register_state(self, **kwargs):
        self.state.register(**kwargs)

    def update_iteration(self, iteration):
        self.state.iteration = iteration

    def save_checkpoint(self, path):
        self.logger.info("Saving checkpoint to file {}".format(path))
        t_start = time.time()

        state_dict = {}
        new_state_dict = OrderedDict()

        for k, v in self.state.model.state_dict().items():
            key = k
            if k.split('.')[0] == 'module':
                key = k[7:]
            new_state_dict[key] = v
        state_dict['model'] = new_state_dict

        if self.state.optimizer:
            state_dict['optimizer'] = self.state.optimizer.state_dict()
        if self.state.scheduler:
            state_dict['scheduler'] = self.state.scheduler.state_dict()
        if self.state.iteration:
            state_dict['iteration'] = self.state.iteration

        t_io_begin = time.time()
        torch.save(state_dict, path)
        t_end = time.time()

        del state_dict
        del new_state_dict

        self.logger.info(
            "Save checkpoint to file {}, "
            "Time usage:\n\tprepare snapshot: {}, IO: {}".format(
                path, t_io_begin - t_start, t_end - t_io_begin))

    def load_checkpoint(self, weights, is_restore=False):

        t_start = time.time()

        if weights.endswith(".pkl"):
            # for caffe2 model
            from maskrcnn_benchmark.utils.c2_model_loading import \
                load_c2_format
            loaded = load_c2_format(self.cfg, weights)
        else:
            loaded = torch.load(weights, map_location=torch.device("cpu"))

        t_io_end = time.time()
        if "model" not in loaded:
            loaded = dict(model=loaded)

        self.state.model = load_model(
            self.state.model, loaded['model'], self.logger,
            is_restore=is_restore)

        if "optimizer" in loaded:
            self.state.optimizer.load_state_dict(loaded['optimizer'])
        if "iteration" in loaded:
            self.state.iteration = loaded['iteration']
        if "scheduler" in loaded:
            self.state.scheduler.load_state_dict(loaded["scheduler"])
        del loaded

        t_end = time.time()
        self.logger.info(
            "Load checkpoint from file {}, "
            "Time usage:\n\tIO: {}, restore snapshot: {}".format(
                weights, t_io_end - t_start, t_end - t_io_end))

    def save_and_link_checkpoint(self, snapshot_dir):
        ensure_dir(snapshot_dir)
        current_iter_checkpoint = osp.join(
            snapshot_dir, 'iter-{}.pth'.format(self.state.iteration))
        self.save_checkpoint(current_iter_checkpoint)
        last_iter_checkpoint = osp.join(
            snapshot_dir, 'iter-last.pth')
        link_file(current_iter_checkpoint, last_iter_checkpoint)

    def restore_checkpoint(self, is_restore=True):
        self.load_checkpoint(self.continue_state_object, is_restore=is_restore)

    def __exit__(self, type, value, tb):
        torch.cuda.empty_cache()
        if type is not None:
            self.logger.warning(
                "A exception occurred during Engine initialization, "
                "give up running process")
            return False

    def __enter__(self):
        return self

In [35]:
engine = Engine(cfg, custom_parser=parser)
logger = engine.setup_log(name='train', log_dir=cfg.OUTPUT_DIR, file_name='log.txt')

FileNotFoundError: ignored

In [36]:
ensure_dir(cfg.OUTPUT_DIR)
model = MSPN(cfg, run_efficient=cfg.RUN_EFFICIENT)
device = torch.device('cpu')
model.to(device)

MSPN(
  (top): ResNet_top(
    (conv): conv_bn_relu(
      (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
    )
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (stage0): Single_stage_module(
    (downsample): ResNet_downsample_module(
      (layer1): Sequential(
        (0): Bottleneck(
          (conv_bn_relu1): conv_bn_relu(
            (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
            (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (relu): ReLU(inplace=True)
          )
          (conv_bn_relu2): conv_bn_relu(
            (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (relu): ReLU(inp

In [37]:
num_gpu = 1
# default num_gpu: 8, adjust iter settings
cfg.SOLVER.CHECKPOINT_PERIOD = \
        int(cfg.SOLVER.CHECKPOINT_PERIOD * 8 / num_gpu)
cfg.SOLVER.MAX_ITER = int(cfg.SOLVER.MAX_ITER * 8 / num_gpu)
optimizer = make_optimizer(cfg, model, num_gpu)
scheduler = make_lr_scheduler(cfg, optimizer)

engine.register_state(scheduler=scheduler, model=model, optimizer=optimizer)

In [38]:
engine.load_checkpoint(cfg.MODEL.WEIGHT, is_restore=False)

FileNotFoundError: ignored

In [39]:
data_loader = get_train_loader(cfg, num_gpu=num_gpu, is_dist=engine.distributed)

loading annotations into memory...


FileNotFoundError: ignored

In [40]:
max_iter = len(data_loader)
checkpoint_period = cfg.SOLVER.CHECKPOINT_PERIOD
tb_writer = SummaryWriter(cfg.TENSORBOARD_DIR)

NameError: ignored

In [ ]:
model.train()

MSPN(
  (top): ResNet_top(
    (conv): conv_bn_relu(
      (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
    )
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (stage0): Single_stage_module(
    (downsample): ResNet_downsample_module(
      (layer1): Sequential(
        (0): Bottleneck(
          (conv_bn_relu1): conv_bn_relu(
            (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
            (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (relu): ReLU(inplace=True)
          )
          (conv_bn_relu2): conv_bn_relu(
            (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (relu): ReLU(inp

In [41]:
model.train()

MSPN(
  (top): ResNet_top(
    (conv): conv_bn_relu(
      (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
    )
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (stage0): Single_stage_module(
    (downsample): ResNet_downsample_module(
      (layer1): Sequential(
        (0): Bottleneck(
          (conv_bn_relu1): conv_bn_relu(
            (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
            (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (relu): ReLU(inplace=True)
          )
          (conv_bn_relu2): conv_bn_relu(
            (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (relu): ReLU(inp

In [ ]:
for iteration, (images, valids, labels) in enumerate(
        data_loader, engine.state.iteration):
    iteration = iteration + 1
    images = images.to(device)
    valids = valids.to(device)
    labels = labels.to(device)

    scheduler.step()
    loss_dict = model(images, valids, labels)
    losses = sum(loss for loss in loss_dict.values())

    optimizer.zero_grad()
    losses.backward()
    optimizer.step()

    if cfg.RUN_EFFICIENT:
        del images, valids, labels, losses

    if engine.local_rank == 0:
        if iteration % 20 == 0 or iteration == max_iter:
            log_str = 'Iter:%d, LR:%.1e, ' % (
                iteration, optimizer.param_groups[0]["lr"] / num_gpu)
            for key in loss_dict:
                tb_writer.add_scalar(
                    key,  loss_dict[key].mean(), global_step=iteration)
                log_str += key + ': %.3f, ' % float(loss_dict[key])

            time2 = time.time()
            elapsed_time = time2 - time1
            time1 = time2
            required_time = elapsed_time / 20 * (max_iter - iteration)
            hours = required_time // 3600
            mins = required_time % 3600 // 60
            log_str += 'To Finish: %dh%dmin,' % (hours, mins)

            logger.info(log_str)

    if iteration % checkpoint_period == 0 or iteration == max_iter:
        engine.update_iteration(iteration)
        if engine.distributed and (engine.local_rank == 0):
            engine.save_and_link_checkpoint(cfg.OUTPUT_DIR)
        elif not engine.distributed:
            engine.save_and_link_checkpoint(cfg.OUTPUT_DIR)

    if iteration >= max_iter:
        logger.info('Finish training process!')
        break

ValueError: ignored